<a href="https://colab.research.google.com/github/Aswin-Cheerngodan/Text-Generation/blob/main/Text_gen_LSTM__GRUipynb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Importing the dependencies

In [12]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn

In [75]:
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense,SimpleRNN,Embedding,Dropout,BatchNormalization,Input,LSTM,GRU
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.callbacks import EarlyStopping
import tensorflow.keras.utils as ku

Data loading and preprocessing

In [14]:
data = pd.read_csv('/content/drive/MyDrive/Keras,Pytorch,Tensorflow/Text Generator/ArticlesApril2017.csv')
data.head()

,abstract,articleID,articleWordCount,byline,documentType,headline,keywords,multimedia,newDesk,printPage,pubDate,sectionName,snippet,source,typeOfMaterial,webURL
0,NaN,58def1347c459f24986d7c80,716,By STEPHEN HILTNER and SUSAN LEHMAN,article,Finding an Expansive View of a Forgotten Peop...,"['Photography', 'New York Times', 'Niger', 'Fe...",3,Insider,2,2017-04-01 00:15:41,Unknown,One of the largest photo displays in Times his...,The New York Times,News,https://www.nytimes.com/2017/03/31/insider/nig...
1,NaN,58def3237c459f24986d7c84,823,By GAIL COLLINS,article,"And Now, the Dreaded Trump Curse","['United States Politics and Government', 'Tru...",3,OpEd,23,2017-04-01 00:23:58,Unknown,Meet the gang from under the bus.,The New York Times,Op-Ed,https://www.nytimes.com/2017/03/31/opinion/and...
2,NaN,58def9f57c459f24986d7c90,575,By THE EDITORIAL BOARD,article,Venezuela’s Descent Into Dictatorship,"['Venezuela', 'Politics and Government', 'Madu...",3,Editorial,22,2017-04-01 00:53:06,Unknown,A court ruling annulling the legislature’s aut...,The New York Times,Editorial,https://www.nytimes.com/2017/03/31/opinion/ven...
3,NaN,58defd317c459f24986d7c95,1374,By MICHAEL POWELL,article,Stain Permeates Basketball Blue Blood,"['Basketball (College)', 'University of North ...",3,Sports,1,2017-04-01 01:06:52,College Basketball,"For two decades, until 2013, North Carolina en...",The New York Times,News,https://www.nytimes.com/2017/03/31/sports/ncaa...
4,NaN,58df09b77c459f24986d7ca7,708,By DEB AMLEN,article,Taking Things for Granted,['Crossword Puzzles'],3,Games,0,2017-04-01 02:00:14,Unknown,In which Howard Barkin and Will Shortz teach u...,The New York Times,News,https://www.nytimes.com/2017/03/31/crosswords/...


In [15]:
all_headline = list(data['headline'].values)

Data Cleaning

In [16]:
!pip install contractions

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 289.9/289.9 kB 16.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.7/110.7 kB 9.4 MB/s eta 0:00:00


In [17]:
import re
import contractions

In [18]:
def clean_txt(text):
  text = contractions.fix(text)
  text = re.sub(r'[^a-zA-Z\s]','',text)
  text = text.lower()
  return text

In [19]:
headlines = [clean_txt(text) for text in all_headline]

In [20]:
tokenizer = Tokenizer()

def get_seq_of_toks(corpus):
  tokenizer.fit_on_texts(corpus)
  total_words = len(tokenizer.word_index)+1

  input_sequences = []
  for line in corpus:
    token_list = tokenizer.texts_to_sequences([line])[0]
    for i in range(1,len(token_list)-1):
      n_gram_seq = token_list[:i+1]
      input_sequences.append(n_gram_seq)

  return input_sequences,total_words

In [21]:
input_seq,total_words = get_seq_of_toks(headlines)

In [22]:
def generate_pad_seqs(input_seqs):
  max_len = max([len(x) for x in input_seqs])
  input_seqs = np.array(pad_sequences(input_seqs,maxlen=max_len,padding='pre'))
  predictors,label = input_seqs[:,:-1],input_seqs[:,-1]
  label = ku.to_categorical(label,num_classes=total_words)
  return predictors,label,max_len


In [23]:
predictors,label,max_seq_len = generate_pad_seqs(input_seq)

In [24]:
print(total_words)

2366


In [104]:
model = Sequential([
    Input(shape=(max_seq_len-1,)),
    Embedding(total_words,10),
    GRU(150,return_sequences=True),
    BatchNormalization(),
    Dropout(0.1),
    LSTM(250,return_sequences=True),
    BatchNormalization(),
    Dropout(0.1),
    GRU(500,return_sequences=True),
    BatchNormalization(),
    Dropout(0.1),
    LSTM(200),
    BatchNormalization(),
    Dense(total_words,activation='softmax')
])

In [105]:
model.compile(loss='categorical_crossentropy',optimizer='adam')
model.summary()

Model: "sequential_14"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding_14 (Embedding)             │ (None, 17, 10)              │          23,660 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ gru_9 (GRU)                          │ (None, 17, 150)             │          72,900 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_33               │ (None, 17, 150)             │             600 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_36 (Dropout)                 │ (None, 17, 150)             │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_26 (LSTM)                       │ (None, 17, 250)             │         401,000 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_34               │ (None, 17, 250)             │           1,000 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_37 (Dropout)                 │ (None, 17, 250)             │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ gru_10 (GRU)                         │ (None, 17, 500)             │       1,128,000 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_35               │ (None, 17, 500)             │           2,000 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_38 (Dropout)                 │ (None, 17, 500)             │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_27 (LSTM)                       │ (None, 200)                 │         560,800 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_36               │ (None, 200)                 │             800 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_14 (Dense)                     │ (None, 2366)                │         475,566 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 2,666,326 (10.17 MB)

 Trainable params: 2,664,126 (10.16 MB)

 Non-trainable params: 2,200 (8.59 KB)

In [106]:
model.fit(predictors,label,batch_size=20,epochs=100)

Epoch 1/100
198/198 ━━━━━━━━━━━━━━━━━━━━ 8s 14ms/step - loss: 7.7380
Epoch 2/100
198/198 ━━━━━━━━━━━━━━━━━━━━ 5s 14ms/step - loss: 6.8928
Epoch 3/100
198/198 ━━━━━━━━━━━━━━━━━━━━ 6s 20ms/step - loss: 6.1633
Epoch 4/100
198/198 ━━━━━━━━━━━━━━━━━━━━ 4s 14ms/step - loss: 5.7572
Epoch 5/100
198/198 ━━━━━━━━━━━━━━━━━━━━ 3s 13ms/step - loss: 5.3435
Epoch 6/100
198/198 ━━━━━━━━━━━━━━━━━━━━ 6s 20ms/step - loss: 4.9079
Epoch 7/100
198/198 ━━━━━━━━━━━━━━━━━━━━ 3s 16ms/step - loss: 4.4304
Epoch 8/100
198/198 ━━━━━━━━━━━━━━━━━━━━ 5s 16ms/step - loss: 3.9758
Epoch 9/100
198/198 ━━━━━━━━━━━━━━━━━━━━ 5s 18ms/step - loss: 3.5594
Epoch 10/100
198/198 ━━━━━━━━━━━━━━━━━━━━ 3s 16ms/step - loss: 3.0985
Epoch 11/100
198/198 ━━━━━━━━━━━━━━━━━━━━ 5s 14ms/step - loss: 2.7568
Epoch 12/100
198/198 ━━━━━━━━━━━━━━━━━━━━ 3s 13ms/step - loss: 2.3662
Epoch 13/100
198/198 ━━━━━━━━━━━━━━━━━━━━ 6s 16ms/step - loss: 1.9701
Epoch 14/100
198/198 ━━━━━━━━━━━━━━━━━━━━ 3s 13ms/step - loss: 1.7307
Epoch 15/100
198/198 ━━━━━━━━

In [107]:
def generate_text(seed_text, next_words, model, max_sequence_len):
    for _ in range(next_words):
        token_list = tokenizer.texts_to_sequences([seed_text])[0]
        token_list = pad_sequences([token_list],
        maxlen=max_sequence_len-1, padding='pre')
        predicted = model.predict(token_list, verbose=0)

        output_word = ""
        for word,index in tokenizer.word_index.items():
            if index ==np.argmax(predicted):
                output_word = word
                break
        seed_text += " "+output_word
    return seed_text.title()

In [108]:
print (generate_text("india and pakistan", 10, model, max_seq_len))
print (generate_text("president trump", 3, model, max_seq_len))
print (generate_text("united states", 4, model, max_seq_len))
print (generate_text("donald trump", 2, model, max_seq_len))
print (generate_text("new york", 3, model, max_seq_len))
print (generate_text("science and technology", 5, model, max_seq_len))

India And Pakistan Mindful Free Shut By Trump Score A Day At French
President Trump To Your Nokill
United States Further Steps To Move
Donald Trump Still An
New York Today How To
Science And Technology Used To Provide My Amnesiac
